# Competiton metric calculator

> The challenge uses the standard [PASCAL VOC 2010 mean Average Precision (mAP)](http://host.robots.ox.ac.uk/pascal/VOC/voc2010/devkit_doc_08-May-2010.pdf) at IoU > 0.4.

In [1]:
# !pip install pycocotools

In [2]:
import pandas as pd
import numpy as np

# import plotly.express as px
# import plotly.graph_objects as go

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [3]:
class VinBigDataEval:
    """Helper class for calculating the competition metric.
    
    You should remove the duplicated annoatations from the `true_df` dataframe
    before using this script. Otherwise it may give incorrect results.

        >>> vineval = VinBigDataEval(valid_df)
        >>> cocoEvalResults = vineval.evaluate(pred_df)

    Arguments:
        true_df: pd.DataFrame Clean (no duplication) Training/Validating dataframe.

    Authors:
        Peter (https://kaggle.com/pestipeti)

    See:
        https://www.kaggle.com/pestipeti/competition-metric-map-0-4

    Returns: None
    
    """
    def __init__(self, true_df):
        
        self.true_df = true_df

        self.image_ids = true_df["image_id"].unique()
        self.annotations = {
            "type": "instances",
            "images": self.__gen_images(self.image_ids),
            "categories": self.__gen_categories(self.true_df),
            "annotations": self.__gen_annotations(self.true_df, self.image_ids)
        }
        
        self.predictions = {
            "images": self.annotations["images"].copy(),
            "categories": self.annotations["categories"].copy(),
            "annotations": None
        }
        
    def __gen_categories(self, df):
        print("Generating category data...")
        
        if "class_name" not in df.columns:
            df["class_name"] = df["class_id"]
        
        cats = df[["class_name", "class_id"]]
        cats = cats.drop_duplicates().sort_values(by='class_id').values
        
        results = []
        
        for cat in cats:
            results.append({
                "id": cat[1],
                "name": cat[0],
                "supercategory": "none",
            })
            
        return results

    def __gen_images(self, image_ids):
        print("Generating image data...")
        results = []

        for idx, image_id in enumerate(image_ids):

            # Add image identification.
            results.append({
                "id": idx,
            })
            
        return results
    
    def __gen_annotations(self, df, image_ids):
        print("Generating annotation data...")
        k = 0
        results = []
        
        for idx, image_id in enumerate(image_ids):

            # Add image annotations
            for i, row in df[df["image_id"] == image_id].iterrows():

                results.append({
                    "id": k,
                    "image_id": idx,
                    "category_id": row["class_id"],
                    "bbox": np.array([
                        row["x_min"],
                        row["y_min"],
                        row["x_max"],
                        row["y_max"]]
                    ),
                    "segmentation": [],
                    "ignore": 0,
                    "area":(row["x_max"] - row["x_min"]) * (row["y_max"] - row["y_min"]),
                    "iscrowd": 0,
                })

                k += 1
                
        return results

    def __decode_prediction_string(self, pred_str):
        data = list(map(float, pred_str.split(" ")))
        data = np.array(data)

        return data.reshape(-1, 6)    
    
    def __gen_predictions(self, df, image_ids):
        print("Generating prediction data...")
        k = 0
        results = []
        
        for i, row in df.iterrows():
            
            image_id = row["image_id"]
            preds = self.__decode_prediction_string(row["PredictionString"])

            for j, pred in enumerate(preds):

                results.append({
                    "id": k,
                    "image_id": int(np.where(image_ids == image_id)[0]),
                    "category_id": int(pred[0]),
                    "bbox": np.array([
                        pred[2], pred[3], pred[4], pred[5]
                    ]),
                    "segmentation": [],
                    "ignore": 0,
                    "area": (pred[4] - pred[2]) * (pred[5] - pred[3]),
                    "iscrowd": 0,
                    "score": pred[1]
                })

                k += 1
                
        return results
                
    def evaluate(self, pred_df, n_imgs = -1):
        """Evaluating your results
        
        Arguments:
            pred_df: pd.DataFrame your predicted results in the
                     competition output format.

            n_imgs:  int Number of images use for calculating the
                     result.All of the images if `n_imgs` <= 0
                     
        Returns:
            COCOEval object
        """
        
        if pred_df is not None:
            self.predictions["annotations"] = self.__gen_predictions(pred_df, self.image_ids)

        coco_ds = COCO()
        coco_ds.dataset = self.annotations
        coco_ds.createIndex()
        
        coco_dt = COCO()
        coco_dt.dataset = self.predictions
        coco_dt.createIndex()
        
        imgIds=sorted(coco_ds.getImgIds())
        
        if n_imgs > 0:
            imgIds = np.random.choice(imgIds, n_imgs)

        cocoEval = COCOeval(coco_ds, coco_dt, 'bbox')
        cocoEval.params.imgIds  = imgIds
        cocoEval.params.useCats = True
        cocoEval.params.iouType = "bbox"
        cocoEval.params.iouThrs = np.array([0.4])

        cocoEval.evaluate()
        cocoEval.accumulate()
        cocoEval.summarize()
        
        return cocoEval

# Usage

In [4]:
# df = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
# df.fillna(0, inplace=True)
# df.loc[df["class_id"] == 14, ['x_max', 'y_max']] = 1.0
#
# df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../input/vinbigdata-chest-xray-abnormalities-detection/train.csv'

In [5]:
# Removing duplications! DO NOT USE THIS in your training!!!
# df = df.groupby(by=['image_id', 'class_id']).first().reset_index()

In [6]:
# You only need to run this once.
# 1vineval = VinBigDataEval(df)

Generating image data...
Generating category data...
Generating annotation data...


In [7]:
# You will (hopefully) have better predictions
# after every training epochs.
#
# The format is the same as we have in the submission.csv
pred_df = df[["image_id"]]
pred_df = pred_df.drop_duplicates()
pred_df["PredictionString"] = "14 1.0 0 0 1 1"
pred_df.reset_index(drop=True, inplace=True)

pred_df.head()

,image_id,PredictionString
0,000434271f63a053c4128a0ba6352c7f,14 1.0 0 0 1 1
1,00053190460d56c53cc3e57321387478,14 1.0 0 0 1 1
2,0005e8e3701dfb1dd93d53e2ff537b6e,14 1.0 0 0 1 1
3,0006e0a85696f6bb578e84fafa9a5607,14 1.0 0 0 1 1
4,0007d316f756b3fa0baea2ff514ce945,14 1.0 0 0 1 1


In [8]:
# You should evaluate after every n epochs.
cocoEvalRes = vineval.evaluate(pred_df)

Generating prediction data...
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=13.41s).
Accumulating evaluation results...
DONE (t=2.74s).
 Average Precision  (AP) @[ IoU=0.40:0.40 | area=   all | maxDets=100 ] = 0.047
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.40:0.40 | area= small | maxDets=100 ] = 0.117
 Average Precision  (AP) @[ IoU=0.40:0.40 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.40:0.40 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.40:0.40 | area=   all | maxDets=  1 ] = 0.067
 Average Recall     (AR) @[ IoU=0.40:0.40 | area=   all | maxDets= 10 ] = 0.067
 Average Recall     (AR) @[ IoU=0.40:0.40 | area=   all | maxDets=100 ] = 0.067
 Average Recall     (AR) @[ IoU=0.40:0.40 | area= 

In [9]:
cocoEvalRes.stats

array([ 0.04687976, -1.        , -1.        ,  0.11719939,  0.        ,
        0.        ,  0.06666038,  0.06666038,  0.06666038,  0.16665095,
        0.        ,  0.        ])

#### Recalculating with random samples

In [10]:
%%capture
stats = []

# Recalculate the validation score using randomly selected images
for i in range(100):
    cocoEvalRes = vineval.evaluate(pred_df = None, n_imgs = 300)
    stats.append(cocoEvalRes.stats[0])
    
avg = np.array(stats).mean()

In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[x for x in range(len(stats))], y=stats, mode="markers", name="Stats"))
fig.add_trace(go.Scatter(x=[0, 100], y=[avg, avg], mode="lines", name="Mean"))
fig.add_trace(go.Scatter(x=[0, 100], y=[0.052, 0.052], mode="lines", name="Public Baseline"))

fig.update_yaxes(
    range=[0.03, 0.07]
)

fig.update_layout(title='Results of mAP@0.4 (randomly selected 300 images)',
                  yaxis_title='Score',
                  xaxis_title='')

fig.show()